In [1]:
pip install boto3 tensorflow

Note: you may need to restart the kernel to use updated packages.


In [5]:
import boto3
import numpy as np
from PIL import Image
import io

# Bucket name and prefix (directory)
bucket_name = 'obligatoriomlprodmmmbfm'
prefix = 'images/'

# Create an S3 client
s3_client = boto3.client('s3')

# List objects in the bucket under the specified prefix
response = s3_client.list_objects_v2(Bucket=bucket_name, Prefix=prefix)

# Filter image keys (assuming they are .jpg or .png)
image_keys = [obj['Key'] for obj in response.get('Contents', []) if obj['Key'].endswith(('.jpg', '.png'))]

# Function to read an image from S3
def read_image_from_s3(bucket, key):
    s3_response_object = s3_client.get_object(Bucket=bucket, Key=key)
    object_content = s3_response_object['Body'].read()
    return Image.open(io.BytesIO(object_content))


NoCredentialsError: Unable to locate credentials

In [6]:
import os
import boto3
from io import BytesIO
from PIL import Image
import numpy as np

# Function to list and download images from S3
def download_images_from_s3(bucket_name, prefix, s3_client):
    try:
        response = s3_client.list_objects_v2(Bucket=bucket_name, Prefix=prefix)
        images = []
        for obj in response.get('Contents', []):
            key = obj['Key']
            if key.endswith(('png', 'jpg', 'jpeg')):
                img_obj = s3_client.get_object(Bucket=bucket_name, Key=key)
                img_data = img_obj['Body'].read()
                img = Image.open(BytesIO(img_data))
                images.append(img)
        return images
    except boto3.exceptions.Boto3Error as e:
        print(f"Error: {e}")
        return []

# Initialize boto3 client
session = initialize_boto3_session()
s3_client = session.client('s3')

bucket_name = 'obligatoriomlprodmmmbfm'
prefix = 'images/'

# Download and preprocess images
images = download_images_from_s3(bucket_name, prefix, s3_client)
if images:
    images = [img.resize((32, 32)) for img in images]  # Resize to 32x32
    images_array = np.array([np.array(img) for img in images])  # Convert to numpy array
    images_array = images_array / 255.0  # Normalize

    # Labels can be generated or loaded similarly (placeholder for actual labels)
    labels = np.array([0] * len(images))  # Replace with actual labels

    print("Images shape:", images_array.shape)
    print("Labels shape:", labels.shape)
else:
    print("No images downloaded.")

ClientError: An error occurred (ExpiredToken) when calling the ListObjectsV2 operation: The provided token has expired.